In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import os
import sys
import numpy as np
import torch
import gpytorch
import matplotlib.pyplot as plt
import pandas as pd
import xarray as xr
import tqdm
import joblib
import utils as utils
from sklearn.linear_model import LinearRegression

base_dir = os.path.join(os.getcwd(), '..')
sys.path.append(base_dir)

import fit_spatial_Plain_GP as plaingp
import fit_spatial_FaIRGP as fairgp
from src.models.utils_spatial import compute_means
from src.evaluation.metrics import spearman_correlation

In [4]:
def weighted_mean(x, wlat):
    mu = torch.sum(x * wlat, dim=(1, 2)) / (x.size(2) * wlat.sum())
    return mu

def compute_deterministic_metrics(prediction, groundtruth, wlat):
    # Compute raw distances metrics
    difference = prediction.sub(groundtruth)
    mean_bias = weighted_mean(difference, wlat).mean()
    rmse = weighted_mean(torch.square(difference), wlat).mean().sqrt()
    mae = weighted_mean(torch.abs(difference), wlat).mean()

    # Compute spearman correlation
    corr = spearman_correlation(prediction.flatten(), groundtruth.flatten())

    # Encapsulate results in output dictionnary
    output = {'mb': mean_bias.item(),
              'rmse': rmse.item(),
              'mae': mae.item(),
              'corr': corr}
    return output

def compute_probabilistic_metrics(prediction, groundtruth, wlat):
    ll = weighted_mean(prediction.log_prob(groundtruth), wlat).mean()
    lb, ub = prediction.icdf(torch.tensor(0.025)), prediction.icdf(torch.tensor(0.975))
    mask = (groundtruth >= lb) & (groundtruth <= ub)
    calib95 = weighted_mean(mask.float(), wlat).mean()
    
    mu, sigma = prediction.mean, prediction.stddev
    y = (groundtruth - mu) / sigma
    norm = torch.distributions.Normal(0, 1)
    crps = sigma * (y * (2 * norm.cdf(y) - 1) + 2 * norm.log_prob(y).exp() - 1 / np.sqrt(np.pi))
    crps = weighted_mean(crps, wlat).mean()
    
    output = {'ll': ll.item(),
              'calib95': calib95.item(),
              'CRPS': crps.item()}
    return output

In [5]:
ssps = ['SSP126', 'SSP245', 'SSP370', 'SSP585']

## FaIR pattern scaling

### SSP126

In [5]:
train_cfg = {'dataset' : {'dirpath': '../data/', 'keys': ['historical', 'ssp245', 'ssp370', 'ssp585']}}
train_data = plaingp.make_data(train_cfg)

test_cfg = {'dataset' : {'dirpath': '../data/', 'keys': ['ssp126']}}
test_data = plaingp.make_data(test_cfg)

In [6]:
scenarios = train_data.scenarios
all_tas = scenarios.tas
wlat = torch.cos(torch.deg2rad(scenarios[0].lat)).clip(min=torch.finfo(torch.float64).eps)[:, None]
glob_tas = weighted_mean(all_tas, wlat)

pattern_scaling = LinearRegression()
pattern_scaling.fit(glob_tas[:, None], all_tas.reshape(all_tas.size(0), -1))

LinearRegression()

In [7]:
time_slice = slice(-21, None)

pred = compute_means(test_data.scenarios, pattern_scaling)['ssp126']
metrics_ssp126 = compute_deterministic_metrics(pred[time_slice],
                                               test_data.scenarios[0].tas[time_slice],
                                               wlat)

### SSP245

In [8]:
train_cfg = {'dataset' : {'dirpath': '../data/', 'keys': ['historical', 'ssp126', 'ssp370', 'ssp585']}}
train_data = plaingp.make_data(train_cfg)

test_cfg = {'dataset' : {'dirpath': '../data/', 'keys': ['ssp245']}}
test_data = plaingp.make_data(test_cfg)

In [9]:
scenarios = train_data.scenarios
all_tas = scenarios.tas
wlat = torch.cos(torch.deg2rad(scenarios[0].lat)).clip(min=torch.finfo(torch.float64).eps)[:, None]
glob_tas = weighted_mean(all_tas, wlat)

pattern_scaling = LinearRegression()
pattern_scaling.fit(glob_tas[:, None], all_tas.reshape(all_tas.size(0), -1))

LinearRegression()

In [10]:
time_slice = slice(-21, None)

pred = compute_means(test_data.scenarios, pattern_scaling)['ssp245']
metrics_ssp245 = compute_deterministic_metrics(pred[time_slice],
                                               test_data.scenarios[0].tas[time_slice],
                                               wlat)

### SSP370

In [11]:
train_cfg = {'dataset' : {'dirpath': '../data/', 'keys': ['historical', 'ssp126', 'ssp245', 'ssp585']}}
train_data = plaingp.make_data(train_cfg)

test_cfg = {'dataset' : {'dirpath': '../data/', 'keys': ['ssp370']}}
test_data = plaingp.make_data(test_cfg)

In [12]:
scenarios = train_data.scenarios
all_tas = scenarios.tas
wlat = torch.cos(torch.deg2rad(scenarios[0].lat)).clip(min=torch.finfo(torch.float64).eps)[:, None]
glob_tas = weighted_mean(all_tas, wlat)

pattern_scaling = LinearRegression()
pattern_scaling.fit(glob_tas[:, None], all_tas.reshape(all_tas.size(0), -1))

LinearRegression()

In [13]:
time_slice = slice(-21, None)

pred = compute_means(test_data.scenarios, pattern_scaling)['ssp370']
metrics_ssp370 = compute_deterministic_metrics(pred[time_slice],
                                               test_data.scenarios[0].tas[time_slice],
                                               wlat)

### SSP585

In [6]:
train_cfg = {'dataset' : {'dirpath': '../data/', 'keys': ['historical', 'ssp126', 'ssp245', 'ssp370']}, 'response_var': 'pr'}
train_data = plaingp.make_data(train_cfg)

test_cfg = {'dataset' : {'dirpath': '../data/', 'keys': ['ssp585']}, 'response_var': 'pr'}
test_data = plaingp.make_data(test_cfg)

In [7]:
scenarios = train_data.scenarios
all_tas = scenarios.response_var
wlat = torch.cos(torch.deg2rad(scenarios[0].lat)).clip(min=torch.finfo(torch.float64).eps)[:, None]
glob_tas = weighted_mean(all_tas, wlat)

pattern_scaling = LinearRegression()
pattern_scaling.fit(glob_tas[:, None], all_tas.reshape(all_tas.size(0), -1))

LinearRegression()

In [8]:
time_slice = slice(-21, None)

pred = compute_means(test_data.scenarios, pattern_scaling)['ssp585']
metrics_ssp585 = compute_deterministic_metrics(pred[time_slice],
                                               test_data.scenarios[0].response_var[time_slice],
                                               wlat)

In [9]:
#metrics_pattern_scaling = [metrics_ssp126, metrics_ssp245, metrics_ssp370, metrics_ssp585]
metrics_pattern_scaling = [metrics_ssp585]

scores_pattern_scaling = pd.DataFrame(metrics_pattern_scaling)
#scores_pattern_scaling.index = ssps

scores_pattern_scaling.index = ['SSP585']
scores_pattern_scaling.index.name = 'SSP'

In [10]:
#metrics_pattern_scaling = [metrics_ssp126, metrics_ssp245, metrics_ssp370, metrics_ssp585]
metrics_pattern_scaling = [metrics_ssp585]
scores_pattern_scaling = pd.DataFrame(metrics_pattern_scaling)
#scores_pattern_scaling.index = ssps
scores_pattern_scaling.index = ['SSP585']
scores_pattern_scaling.index.name = 'SSP'

In [11]:
#scores_pattern_scaling
metrics_pattern_scaling = [ metrics_ssp585]
scores_pattern_scaling = pd.DataFrame(metrics_pattern_scaling)
scores_pattern_scaling.index = ["SSP585"]
scores_pattern_scaling.index.name = 'SSP'

## PlainGP

### SSP126

In [39]:
train_cfg = {'dataset' : {'dirpath': '../data/', 'keys': ['historical', 'ssp245', 'ssp370', 'ssp585']}}
train_data = plaingp.make_data(train_cfg)

test_cfg = {'dataset' : {'dirpath': '../data/', 'keys': ['ssp126']}}
test_data = plaingp.make_data(test_cfg)

model = plaingp.make_model(train_cfg, train_data)
state_dict = torch.load('../data/models/leave-one-out-ssp/ssp126/SpatialPlainGP/state_dict.pt')
model.load_state_dict(state_dict)

KeyError: 'response_var'

In [21]:
test_scenarios = test_data.scenarios
Xtest = test_scenarios.glob_inputs[:, 1:]
Xtest = (Xtest - model.mu) / model.sigma
test_tas = test_scenarios.tas
model = model.eval()

In [22]:
with torch.no_grad():
    posterior = model.posterior(Xtest, diag=False)
    posterior = model.likelihood(posterior)

In [23]:
nlat = len(test_scenarios[0].lat)
nlon = len(test_scenarios[0].lon)
ntimes = len(test_scenarios[0].timesteps)
time_slice = slice(-21, None)

prior_mean = model.mu_targets.view(1, nlat, nlon).repeat(ntimes, 1, 1)
posterior_mean = posterior.mean.reshape(nlat, nlon, -1).permute(2, 0, 1)
posterior_correction =  model.sigma_targets.view(1, nlat, nlon) * posterior_mean
posterior_mean = prior_mean + posterior_mean

posterior_stddev = posterior.stddev.reshape(nlat, nlon, -1).permute(2, 0, 1)
posterior_stddev = model.sigma_targets.view(1, nlat, nlon) * posterior_stddev

prediction = torch.distributions.Normal(posterior_mean[time_slice], posterior_stddev[time_slice])
groundtruth = test_tas[time_slice]
wlat = torch.cos(torch.deg2rad(test_scenarios[0].lat)).clip(min=torch.finfo(torch.float64).eps)[:, None]

In [24]:
metrics_ssp126 = {**compute_deterministic_metrics(prediction.mean, groundtruth, wlat),
                  **compute_probabilistic_metrics(prediction, groundtruth, wlat)}

### SSP245

In [25]:
train_cfg = {'dataset' : {'dirpath': '../data/', 'keys': ['historical', 'ssp126', 'ssp370', 'ssp585']}}
train_data = plaingp.make_data(train_cfg)

test_cfg = {'dataset' : {'dirpath': '../data/', 'keys': ['ssp245']}}
test_data = plaingp.make_data(test_cfg)

model = plaingp.make_model(train_cfg, train_data)
state_dict = torch.load('../data/models/leave-one-out-ssp/ssp245/SpatialPlainGP/state_dict.pt')
model.load_state_dict(state_dict)

<All keys matched successfully>

In [26]:
test_scenarios = test_data.scenarios
Xtest = test_scenarios.glob_inputs[:, 1:]
Xtest = (Xtest - model.mu) / model.sigma
test_tas = test_scenarios.tas
model = model.eval()

In [27]:
with torch.no_grad():
    posterior = model.posterior(Xtest, diag=False)
    posterior = model.likelihood(posterior)

In [28]:
nlat = len(test_scenarios[0].lat)
nlon = len(test_scenarios[0].lon)
ntimes = len(test_scenarios[0].timesteps)
time_slice = slice(-21, None)

prior_mean = model.mu_targets.view(1, nlat, nlon).repeat(ntimes, 1, 1)
posterior_mean = posterior.mean.reshape(nlat, nlon, -1).permute(2, 0, 1)
posterior_correction =  model.sigma_targets.view(1, nlat, nlon) * posterior_mean
posterior_mean = prior_mean + posterior_mean

posterior_stddev = posterior.stddev.reshape(nlat, nlon, -1).permute(2, 0, 1)
posterior_stddev = model.sigma_targets.view(1, nlat, nlon) * posterior_stddev

prediction = torch.distributions.Normal(posterior_mean[time_slice], posterior_stddev[time_slice])
groundtruth = test_tas[time_slice]
wlat = torch.cos(torch.deg2rad(test_scenarios[0].lat)).clip(min=torch.finfo(torch.float64).eps)[:, None]

In [29]:
metrics_ssp245 = {**compute_deterministic_metrics(prediction.mean, groundtruth, wlat),
                  **compute_probabilistic_metrics(prediction, groundtruth, wlat)}

### SSP370

In [31]:
train_cfg = {'dataset' : {'dirpath': '../data/', 'keys': ['historical', 'ssp126', 'ssp245', 'ssp585']}}
train_data = plaingp.make_data(train_cfg)

test_cfg = {'dataset' : {'dirpath': '../data/', 'keys': ['ssp370']}}
test_data = plaingp.make_data(test_cfg)

model = plaingp.make_model(train_cfg, train_data)
state_dict = torch.load('../data/models/leave-one-out-ssp/ssp370/SpatialPlainGP/state_dict.pt')
model.load_state_dict(state_dict)

<All keys matched successfully>

In [32]:
test_scenarios = test_data.scenarios
Xtest = test_scenarios.glob_inputs[:, 1:]
Xtest = (Xtest - model.mu) / model.sigma
test_tas = test_scenarios.tas
model = model.eval()

In [33]:
with torch.no_grad():
    posterior = model.posterior(Xtest, diag=False)
    posterior = model.likelihood(posterior)

In [34]:
nlat = len(test_scenarios[0].lat)
nlon = len(test_scenarios[0].lon)
ntimes = len(test_scenarios[0].timesteps)
time_slice = slice(-21, None)

prior_mean = model.mu_targets.view(1, nlat, nlon).repeat(ntimes, 1, 1)
posterior_mean = posterior.mean.reshape(nlat, nlon, -1).permute(2, 0, 1)
posterior_correction =  model.sigma_targets.view(1, nlat, nlon) * posterior_mean
posterior_mean = prior_mean + posterior_mean

posterior_stddev = posterior.stddev.reshape(nlat, nlon, -1).permute(2, 0, 1)
posterior_stddev = model.sigma_targets.view(1, nlat, nlon) * posterior_stddev

prediction = torch.distributions.Normal(posterior_mean[time_slice], posterior_stddev[time_slice])
groundtruth = test_tas[time_slice]
wlat = torch.cos(torch.deg2rad(test_scenarios[0].lat)).clip(min=torch.finfo(torch.float64).eps)[:, None]

In [35]:
metrics_ssp370 = {**compute_deterministic_metrics(prediction.mean, groundtruth, wlat),
                  **compute_probabilistic_metrics(prediction, groundtruth, wlat)}

### SSP585

In [12]:
train_cfg = {'dataset' : {'dirpath': '../data/', 'keys': ['historical', 'ssp126', 'ssp245', 'ssp370']} , 'response_var': 'pr'}
train_data = plaingp.make_data(train_cfg)

test_cfg = {'dataset' : {'dirpath': '../data/', 'keys': ['ssp585']} , 'response_var': 'pr'}
test_data = plaingp.make_data(test_cfg)

model = plaingp.make_model(train_cfg, train_data)
#state_dict = torch.load('../data/models/leave-one-out-ssp/ssp585/SpatialPlainGP/state_dict.pt')
state_dict = torch.load('../spatial_plain_gp_output/state_dict.pt')
model.load_state_dict(state_dict)

<All keys matched successfully>

In [13]:
test_scenarios = test_data.scenarios
Xtest = test_scenarios.glob_inputs[:, 1:]
Xtest = (Xtest - model.mu) / model.sigma
test_tas = test_scenarios.response_var
model = model.eval()

In [14]:
with torch.no_grad():
    posterior = model.posterior(Xtest, diag=False)
    posterior = model.likelihood(posterior)

In [15]:
nlat = len(test_scenarios[0].lat)
nlon = len(test_scenarios[0].lon)
ntimes = len(test_scenarios[0].timesteps)
time_slice = slice(-21, None)

prior_mean = model.mu_targets.view(1, nlat, nlon).repeat(ntimes, 1, 1)
posterior_mean = posterior.mean.reshape(nlat, nlon, -1).permute(2, 0, 1)
posterior_correction =  model.sigma_targets.view(1, nlat, nlon) * posterior_mean
posterior_mean = prior_mean + posterior_mean

posterior_stddev = posterior.stddev.reshape(nlat, nlon, -1).permute(2, 0, 1)
posterior_stddev = model.sigma_targets.view(1, nlat, nlon) * posterior_stddev

prediction = torch.distributions.Normal(posterior_mean[time_slice], posterior_stddev[time_slice])
groundtruth = test_tas[time_slice]
wlat = torch.cos(torch.deg2rad(test_scenarios[0].lat)).clip(min=torch.finfo(torch.float64).eps)[:, None]

In [16]:
metrics_ssp585 = {**compute_deterministic_metrics(prediction.mean, groundtruth, wlat),
                  **compute_probabilistic_metrics(prediction, groundtruth, wlat)}

In [17]:
ssps

['SSP126', 'SSP245', 'SSP370', 'SSP585']

In [18]:
#metrics_plaingp = [metrics_ssp126, metrics_ssp245, metrics_ssp370, metrics_ssp585]
metrics_plaingp = [metrics_ssp585]
scores_plaingp = pd.DataFrame(metrics_plaingp)
#scores_plaingp.index = ssps
scores_plaingp.index = ['SSP585']
scores_plaingp.index.name = 'SSP'
scores_plaingp

,mb,rmse,mae,corr,ll,calib95,CRPS
SSP,,,,,,,
SSP585,0.118038,0.477741,0.381884,0.335349,-1.653220e+11,0.000018,0.381881


# FaIRGP

### SSP126

In [42]:
train_cfg = {'dataset' : {'dirpath': '../data/', 'keys': ['historical', 'ssp245', 'ssp370', 'ssp585']}}
train_data = fairgp.make_data(train_cfg)

test_cfg = {'dataset' : {'dirpath': '../data/', 'keys': ['ssp126']}}
test_data = fairgp.make_data(test_cfg)

model = fairgp.make_model(train_cfg, train_data)
state_dict = torch.load('../data/models/leave-one-out-ssp/ssp126/SpatialFaIRGP/state_dict.pt')
model.load_state_dict(state_dict)

<All keys matched successfully>

In [43]:
test_scenarios = test_data.scenarios
test_tas = test_scenarios.tas

model = model.eval()
with torch.no_grad():
    posterior = model.posterior(test_scenarios, diag=False)
    posterior = model.likelihood(posterior)

In [44]:
nlat = len(test_scenarios[0].lat)
nlon = len(test_scenarios[0].lon)
time_slice = slice(-21, None)

prior_mean = model._compute_means(test_scenarios)['ssp126']
posterior_mean = posterior.mean.reshape(nlat, nlon, -1).permute(2, 0, 1)
posterior_correction = model.mu_targets + model.sigma_targets * posterior_mean
posterior_correction = model.beta.unsqueeze(0) * posterior_correction
posterior_mean = prior_mean + posterior_correction

posterior_stddev = posterior.stddev.reshape(nlat, nlon, -1).permute(2, 0, 1)
posterior_stddev = model.sigma_targets * posterior_stddev * torch.abs(model.beta.unsqueeze(0))

prediction = torch.distributions.Normal(posterior_mean[time_slice], posterior_stddev[time_slice])
groundtruth = test_tas[time_slice]

In [45]:
metrics_ssp126 = {**compute_deterministic_metrics(prediction.mean, groundtruth, wlat),
                  **compute_probabilistic_metrics(prediction, groundtruth, wlat)}

### SSP245

In [46]:
train_cfg = {'dataset' : {'dirpath': '../data/', 'keys': ['historical', 'ssp126', 'ssp370', 'ssp585']}}
train_data = fairgp.make_data(train_cfg)

test_cfg = {'dataset' : {'dirpath': '../data/', 'keys': ['ssp245']}}
test_data = fairgp.make_data(test_cfg)

model = fairgp.make_model(train_cfg, train_data)
state_dict = torch.load('../data/models/leave-one-out-ssp/ssp245/SpatialFaIRGP/state_dict.pt')
model.load_state_dict(state_dict)

<All keys matched successfully>

In [47]:
test_scenarios = test_data.scenarios
test_tas = test_scenarios.tas

model = model.eval()
with torch.no_grad():
    posterior = model.posterior(test_scenarios, diag=False)
    posterior = model.likelihood(posterior)

In [48]:
nlat = len(test_scenarios[0].lat)
nlon = len(test_scenarios[0].lon)
time_slice = slice(-21, None)

prior_mean = model._compute_means(test_scenarios)['ssp245']
posterior_mean = posterior.mean.reshape(nlat, nlon, -1).permute(2, 0, 1)
posterior_correction = model.mu_targets + model.sigma_targets * posterior_mean
posterior_correction = model.beta.unsqueeze(0) * posterior_correction
posterior_mean = prior_mean + posterior_correction

posterior_stddev = posterior.stddev.reshape(nlat, nlon, -1).permute(2, 0, 1)
posterior_stddev = model.sigma_targets * posterior_stddev * torch.abs(model.beta.unsqueeze(0))

prediction = torch.distributions.Normal(posterior_mean[time_slice], posterior_stddev[time_slice])
groundtruth = test_tas[time_slice]

In [49]:
metrics_ssp245 = {**compute_deterministic_metrics(prediction.mean, groundtruth, wlat),
                  **compute_probabilistic_metrics(prediction, groundtruth, wlat)}

### SSP370

In [50]:
train_cfg = {'dataset' : {'dirpath': '../data/', 'keys': ['historical', 'ssp126', 'ssp245', 'ssp585']}}
train_data = fairgp.make_data(train_cfg)

test_cfg = {'dataset' : {'dirpath': '../data/', 'keys': ['ssp370']}}
test_data = fairgp.make_data(test_cfg)

model = fairgp.make_model(train_cfg, train_data)
state_dict = torch.load('../data/models/leave-one-out-ssp/ssp370/SpatialFaIRGP/state_dict.pt')
model.load_state_dict(state_dict)

<All keys matched successfully>

In [51]:
test_scenarios = test_data.scenarios
test_tas = test_scenarios.tas

model = model.eval()
with torch.no_grad():
    posterior = model.posterior(test_scenarios, diag=False)
    posterior = model.likelihood(posterior)

In [52]:
nlat = len(test_scenarios[0].lat)
nlon = len(test_scenarios[0].lon)
time_slice = slice(-21, None)

prior_mean = model._compute_means(test_scenarios)['ssp370']
posterior_mean = posterior.mean.reshape(nlat, nlon, -1).permute(2, 0, 1)
posterior_correction = model.mu_targets + model.sigma_targets * posterior_mean
posterior_correction = model.beta.unsqueeze(0) * posterior_correction
posterior_mean = prior_mean + posterior_correction

posterior_stddev = posterior.stddev.reshape(nlat, nlon, -1).permute(2, 0, 1)
posterior_stddev = model.sigma_targets * posterior_stddev * torch.abs(model.beta.unsqueeze(0))

prediction = torch.distributions.Normal(posterior_mean[time_slice], posterior_stddev[time_slice])
groundtruth = test_tas[time_slice]

In [53]:
metrics_ssp370 = {**compute_deterministic_metrics(prediction.mean, groundtruth, wlat),
                  **compute_probabilistic_metrics(prediction, groundtruth, wlat)}

### SSP585

In [22]:
train_cfg = {'dataset' : {'dirpath': '../data/', 'keys': ['historical', 'ssp126', 'ssp245', 'ssp370']}, 'response_var': 'pr'}
train_data = fairgp.make_data(train_cfg)

test_cfg = {'dataset' : {'dirpath': '../data/', 'keys': ['ssp585']}, 'response_var': 'pr'}
test_data = fairgp.make_data(test_cfg)

model = fairgp.make_model(train_cfg, train_data)
state_dict = torch.load('../spatial_fairgp_output/state_dict.pt')
model.load_state_dict(state_dict)

<All keys matched successfully>

In [23]:
test_scenarios = test_data.scenarios
#test_tas = test_scenarios.tas
test_tas = test_scenarios.response_var

model = model.eval()
with torch.no_grad():
    posterior = model.posterior(test_scenarios, diag=False)
    posterior = model.likelihood(posterior)

In [24]:
nlat = len(test_scenarios[0].lat)
nlon = len(test_scenarios[0].lon)
time_slice = slice(-21, None)

prior_mean = model._compute_means(test_scenarios)['ssp585']
posterior_mean = posterior.mean.reshape(nlat, nlon, -1).permute(2, 0, 1)
posterior_correction = model.mu_targets + model.sigma_targets * posterior_mean
posterior_correction = model.beta.unsqueeze(0) * posterior_correction
posterior_mean = prior_mean + posterior_correction

posterior_stddev = posterior.stddev.reshape(nlat, nlon, -1).permute(2, 0, 1)
posterior_stddev = model.sigma_targets * posterior_stddev * torch.abs(model.beta.unsqueeze(0))

prediction = torch.distributions.Normal(posterior_mean[time_slice], posterior_stddev[time_slice])
groundtruth = test_tas[time_slice]

In [25]:
metrics_ssp585 = {**compute_deterministic_metrics(prediction.mean, groundtruth, wlat),
                  **compute_probabilistic_metrics(prediction, groundtruth, wlat)}

In [26]:
#metrics_fairgp = [metrics_ssp126, metrics_ssp245, metrics_ssp370, metrics_ssp585]
metrics_fairgp = [metrics_ssp585]
scores_fairgp = pd.DataFrame(metrics_fairgp)
#scores_fairgp.index = ssps
scores_fairgp.index = ['SSP585']
scores_fairgp.index.name = 'SSP'
scores_fairgp

,mb,rmse,mae,corr,ll,calib95,CRPS
SSP,,,,,,,
SSP585,0.489339,4.501308,2.272548,0.389457,-1.989164,0.571431,1.636987


---

In [27]:
pattern_scaling_summary = scores_pattern_scaling.aggregate(['mean', 'std'])
gp_summary = scores_plaingp.aggregate(['mean', 'std'])
fairgp_summary = scores_fairgp.aggregate(['mean', 'std'])

scores_pattern_scaling = pd.concat([scores_pattern_scaling, pattern_scaling_summary])
scores_plaingp = pd.concat([scores_plaingp, gp_summary])
scores_fairgp = pd.concat([scores_fairgp, fairgp_summary])

In [79]:
scores_fairgp['Model'] = '2FaIRGP'
scores_plaingp['Model'] = '1Plain GP'
scores_pattern_scaling['Model'] = '0FaIR'
metrics_cols = ['Bias', 'RMSE', 'MAE', 'Corr', 'LL', 'Calib95', 'CRPS']
full_scores = pd.concat([scores_fairgp, scores_plaingp, scores_pattern_scaling]).sort_index().set_index('Model', append=True)
full_scores.columns = metrics_cols
full_scores = full_scores[['RMSE', 'MAE', 'Bias', 'Corr', 'LL', 'Calib95', 'CRPS']].sort_index().round(3)

In [80]:
results = full_scores[['RMSE', 'MAE', 'Bias', 'LL', 'Calib95', 'CRPS']].fillna('-')
# results.to_latex('./tables/ssp-experiment-spatial-scores.tex')
results

RMSE     MAE   Bias              LL Calib95   CRPS
       Model                                                         
SSP585 0FaIR      27.85  15.255  3.285               -       -      -
       1Plain GP  0.478   0.382  0.118 -165321965568.0     0.0  0.382
       2FaIRGP    4.501   2.273  0.489          -1.989   0.571  1.637
mean   0FaIR      27.85  15.255  3.285               -       -      -
       1Plain GP  0.478   0.382  0.118 -165321965568.0     0.0  0.382
       2FaIRGP    4.501   2.273  0.489          -1.989   0.571  1.637
std    0FaIR          -       -      -               -       -      -
       1Plain GP      -       -      -               -       -      -
       2FaIRGP        -       -      -               -       -      -